In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import soundfile as sf

# for recording in notebooks
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

# for converting audio file
from pydub import AudioSegment

In [11]:
# load models

whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large").to("cuda")
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-large")

In [12]:
# transcription funcion

def whisper(file):
    input_file = sf.read(file)
    sample_rate = sf.info(file).samplerate
    
    input_features = whisper_processor(input_file[0], sampling_rate=sample_rate )
    input_features = torch.tensor( input_features['input_features'] ).to("cuda")
    
    with torch.no_grad():
        generated_ids = whisper_model.generate(inputs=input_features)
    
    transcription = whisper_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return transcription

In [5]:
# recorder in notebook
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [16]:
output_file = 'output.flac'

def transform_recorder_to_flac(output_file):
    recorder.save('output.webm')
    sound_webm = AudioSegment.from_file('output.webm').set_frame_rate(16000)
    sound_webm.export(output_file, format='flac')
    
transform_recorder_to_flac(output_file)
transcription = whisper(output_file)
print(transcription)

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation_utils.py:1296: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 Yes, I know. Isn't it cold?


In [3]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

mname = "facebook/blenderbot-3B"
model = BlenderbotForConditionalGeneration.from_pretrained(mname).to("cuda")
tokenizer = BlenderbotTokenizer.from_pretrained(mname)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
UTTERANCE = transcription
inputs = tokenizer([UTTERANCE], return_tensors="pt")
inputs['input_ids'] = inputs['input_ids'].to('cuda')
inputs['attention_mask'] = inputs['attention_mask'].to('cuda')
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation_utils.py:1296: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 60 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<s> Yes, it is very cold.  I am glad I am not the only one who feels that way.</s>']
